### LIBRERIAS NECESARIAS PARA LAS FUNCIONES

In [24]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import gc

### ARCHIVOS NECESARIOS PARA LAS FUNCIONES

In [13]:
# ARCHIVOS NECESARIOS COMPLETOS. De todas formas en cada función se levantan solo las columnas necesarias para el desarrollo de cada una particularmente

# df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip')
# df_items = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv", sep=';', on_bad_lines='skip')
# df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip')

###  FUNCIONES

##### N° 1 def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

In [21]:
def developer( desarrollador : str ):
    """Cantidad de items y porcentaje de contenido Free 
        por año según empresa desarrolladora"""
     
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['developer', 'year', 'price']) 
        
    #Filtrar juegos por desarrollador
    df_desarrollador = df_steam[df_steam['developer'] == desarrollador].copy()
    
    # Contar la cantidad de items por año
    cantidad_items_por_year = df_desarrollador.groupby('year').size().reset_index(name="cantidad_items")
    
    # Contar la cantidad de items GRATUITOS por año
    cantidad_items_gratuitos_por_year = df_desarrollador[df_desarrollador["price"] == 0].groupby('year').size().reset_index(name='cantidad_items_gratuitos_por_año').astype('Int64')

    # Combinar filtros para dar resultado y creación de columna que calcule el porcentaje sobre el total
    resultado = pd.merge(cantidad_items_por_year, cantidad_items_gratuitos_por_year, on='year', how='left').fillna(0)
    resultado['% Contenido Free'] = ((resultado['cantidad_items_gratuitos_por_año'] / resultado['cantidad_items']) * 100).round(2)
    
    # Convertir el DataFrame a un diccionario
    resultado_dict = resultado.to_dict(orient='records')
    
    return resultado_dict

developer("SmiteWorks USA, LLC") #probando función


[{'year': 2014,
  'cantidad_items': 74,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2015,
  'cantidad_items': 80,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2016,
  'cantidad_items': 231,
  'cantidad_items_gratuitos_por_año': 0,
  '% Contenido Free': 0.0},
 {'year': 2017,
  'cantidad_items': 428,
  'cantidad_items_gratuitos_por_año': 4,
  '% Contenido Free': 0.93}]

#### N° 2 def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. 


In [7]:
# Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

def userdata( User_id : str ):
    """Devuelve cantidad de dinero gastado por el usuario, el porcentaje de 
    recomendación en base a reviews.recommend y cantidad de items"""

    df_items = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id', 'item_id', 'price'])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','recommend'])
    
    #Filtrar por user_id
    df_user = df_items[df_items['user_id']== User_id]

    #Calcular el gasto total por  usuario
    gasto = str(round(df_user['price'].sum(),2))+' USD'

    #Calcular la cantidad de items comprados
    cantidad_items = df_user['item_id'].count()
    
    # Filtrar reviews por ususario 
    df_reviews_usuario = df_reviews[df_reviews['user_id']== User_id]
    
    #Calcular la cantidad de juegos recomendados (True)
    cantidad_recomendaciones = (df_reviews_usuario['recommend'] == True).sum()

    # Calcular la cantidad de juegos que tienen datos en la columna review
    cantidad_juegos_con_review = df_reviews_usuario['recommend'].notna().sum()

    #Porcentaje de recomendación
    if cantidad_items > 0:
        porcentaje_recomendaciones = str(round(cantidad_recomendaciones * 100 / cantidad_juegos_con_review, 2)) + '%'
    else:
        porcentaje_recomendaciones = '0%'

    #Muestra los resultados en un diccionario y con el formato adecuado
    return {"Id Usuario" : str(User_id), 
           "Dinero gastado": str(gasto), 
           "% de recomendación": str(porcentaje_recomendaciones),
           "Cantidad de items": int(cantidad_items)  
           }
userdata("STEAM0082987612")

{'Id Usuario': 'STEAM0082987612',
 'Dinero gastado': '1209.26 USD',
 '% de recomendación': '85.71%',
 'Cantidad de items': 106}

#### N° 3 def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.


In [ ]:
# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UserForGenre( genero : str ): 
    """Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista 
    de la acumulación de horas jugadas por año de lanzamiento."""
    
    df_items = pd.read_parquet("./Bases de datos/Archivos Post ETL/items_post_etl.parquet", columns=['user_id', 'item_id', 'playtime_forever']) 
    df_steam = pd.read_csv("./Bases de datos/Archivos Post ETL/steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"genres"])
  
    genero = genero.lower()
    genero = genero.title()

    # Filtrar el DataFrame df_steam para obtener los juegos del género especificado
    df_steam_filtrado  = df_steam[df_steam['genres'].apply(lambda x: genero in x)]
   
    # Unir df_steam_filtrado con df_items basado en 'item_id' para obtener los usuarios y horas jugadas para los juegos del género 
    df_combinado = pd.merge(df_items[['user_id', 'item_id', 'playtime_forever']], df_steam[['item_id', 'year']], on='item_id', how='inner')
    # df_combinado = pd.merge(df_steam_filtrado[['item_id', 'year',"genres"]], df_items[['user_id', 'item_id', 'playtime_forever']], on='item_id', how='inner')

    # Agrupar por 'user_id' y sumar las horas jugadas para cada usuario
    horas_por_usuario = df_combinado.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Obtener el usuario con más horas jugadas
    usuario_mas_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax(), 'user_id']

    # Agrupar por año y sumar las horas jugadas para cada año
    horas_por_año_usuario = df_combinado[df_combinado['user_id'] == usuario_mas_horas].groupby('year')['playtime_forever'].sum().reset_index()

    # Convertir a la lista deseada de acumulación de horas jugadas por año
    dicc_horas_por_year = [{'Año': row['year'], 'Horas': row['playtime_forever']} for index, row in horas_por_año_usuario.iterrows()]

    respuesta = {
    f"Usuario con más horas jugadas para Género {genero}": usuario_mas_horas,
    "Horas jugadas": dicc_horas_por_year }

    return respuesta

UserForGenre("Sports") 

#### N° 4 def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)


In [11]:
# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}
def best_developer_year( year : int ):
    """Devuelve el top 3 de desarrolladores con juegos 
    MÁS recomendados por usuarios para el año dado. """
    
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"developer"])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','item_id','recommend'])
    
    # Filtrar df_steam por el año dado
    juegos_por_año = df_steam[df_steam['year'] == year]

    # Unir df_steam con df_reviews por item_id
    merged_df = pd.merge(juegos_por_año, df_reviews, on='item_id', how='left')

    # Filtrar solo las recomendaciones positivas
    recomendaciones_positivas = merged_df[merged_df['recommend'] == True]

    # Contar recomendaciones por desarrollador
    conteo_recomendaciones = recomendaciones_positivas.groupby('developer').size().reset_index(name='cantidad_recomendaciones')

    # Ordenar df para obtener los primeros 3 desarrolladores segun la cantidad de recomendaciones
    top_desarrolladores = conteo_recomendaciones.sort_values(by='cantidad_recomendaciones', ascending=False).head(3)

    # Guardo los tres primeros puestos
    primer_puesto = top_desarrolladores.iloc[0]['developer']
    segundo_puesto = top_desarrolladores.iloc[1]['developer']
    tercer_puesto = top_desarrolladores.iloc[2]['developer']

    # Formatear el resultado
    resultado = {
        "Puesto 1": primer_puesto,
        "Puesto 2": segundo_puesto,
        "Puesto 3": tercer_puesto
    }
    return resultado

best_developer_year(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Freejam',
 'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}

#### N° 5 def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.


In [12]:
# Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}
def developer_reviews_analysis( desarrolladora : str ): 
    """Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y 
    una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados 
    con un análisis de sentimiento como valor positivo o negativo."""
    
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id',"developer"])
    df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', on_bad_lines='skip',usecols=['user_id','item_id','sentiment_analysis'])
    
    
    # Filtrar df_steam por la desarrolladora especificada
    juegos_desarrolladora = df_steam[df_steam['developer'] == desarrolladora]
    
    # Unir df_steam con df_reviews por item_id
    merged_df = pd.merge(juegos_desarrolladora, df_reviews, on='item_id', how='left')

    # Contar la cantidad total de reseñas positivas y negativas
    conteo_reseñas = merged_df['sentiment_analysis'].value_counts().to_dict()

        # Crear el diccionario de resultados
    reseñas = {
        desarrolladora: [
            conteo_reseñas.get(2.0, 0),   # Positive count
            conteo_reseñas.get(0.0, 0)      # Negative count
        ]
    }

    # Renombrar claves en la lista a 'Positivas' y 'Negativas'
    reseñas[desarrolladora] = {
        "Positivas": reseñas[desarrolladora][0],
        "Negativas": reseñas[desarrolladora][1]
    }

    return reseñas

developer_reviews_analysis("Team Psykskallar")

{'Team Psykskallar': {'Positivas': 33, 'Negativas': 34}}

###  MODELO ML

Modelo de aprendizaje automático:

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. Deben crear al menos uno de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Creación de un sistema de recomendación item-item con similitud del coseno:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix

def recomendar_juegos_genero(item_id):
    """ Se consulta  un item_id, y se recomiendan 5 juegos basados 
        en su similitud respecto a sus generos respectivos"""
    try:
        # Cargar los dataframes
        df_steam = pd.read_csv(".\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', usecols=['item_id', 'title', 'genres']) 
        # df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', usecols=['item_id', 'sentiment_analysis'])
       
        # Verificar si el item_id existe en el dataframe
        if item_id not in df_steam['item_id'].values:
            raise ValueError(f"El item_id {item_id} no está en la base de datos.")

        # Filtrar el dataframe de Steam para obtener solo los juegos relevantes
        df_steam = df_steam.dropna(subset=['genres'])  # Eliminar filas con NaN en géneros

        # Obtener el género del juego buscado
        game_genres_str = df_steam[df_steam['item_id'] == item_id]['genres'].values[0]
        game_genres = ast.literal_eval(game_genres_str)
        
        # verificar si el juego no tiene géneros válidos
        if not game_genres_str or game_genres_str == '[]':
            raise ValueError(f"El juego con item_id {item_id} no tiene géneros asignados, por lo que no podemos darte una recomendación para géneros similares.")


        # Filtro los juegos que tengan al menos alguno de esos generos y restablecer indices
        filtered_df_steam = df_steam[df_steam['genres'].apply(lambda x: any(genre in eval(x) for genre in game_genres))]
        filtered_df_steam = filtered_df_steam.reset_index(drop=True)

        # Si no hay suficientes juegos similares en el filtro
        if filtered_df_steam.shape[0] <= 1:
            raise ValueError(f"No se encontraron suficientes juegos similares al juego con item_id {item_id}.")

        # Vectorizar la columna 'genres':
        # Obj: Convertir el texto de la columna genres en una representación numérica que pueda ser utilizada para calcular similitudes.
        tfidf = TfidfVectorizer()
        tfidf_matrix = tfidf.fit_transform(filtered_df_steam['genres'])
        
         # Obtener el índice del juego buscado
        idx = filtered_df_steam.index[filtered_df_steam['item_id'] == item_id].tolist()[0]

        # Calcular la similitud coseno entre todos los juegos
        # Obj: Calcular la similitud entre todos los juegos basándose en sus géneros.
        sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
   


        #Calcular las puntuaciones de similitud entre el juego buscado y todos los demás juegos    
        sim_scores_idx = list(enumerate(sim_scores))

        # Ordenar las puntuaciones de similitud en orden descendente (juegos más similares primero)
        sim_scores_idx = sorted(sim_scores_idx, key=lambda x: x[1], reverse=True)

        # Obtener los índices de los juegos más similares (excluyendo el propio juego)
        # Esto se hace porque la primera entrada (sim_scores[0]) es el propio juego, que tendrá una similitud de 1  y no lo queremos en las recomendaciones.
        top_indices = [i[0] for i in sim_scores_idx[1:6]]

        # Retornar los títulos de los juegos recomendados
        return filtered_df_steam.iloc[top_indices][['title', 'genres']]
    except ValueError as e:
        return str(e)  # Retornar el mensaje de error si ocurre
    except Exception as e:
        return f"Ha ocurrido un error inesperado: {str("Lo solucionaremos en breve")}"


In [16]:
recomendar_juegos_genero(15710)

,title,genres
26,Bone: The Great Cow Race,['Adventure']
27,Dracula Origin,['Adventure']
30,Oddworld: Abe's Exoddus®,['Adventure']
31,Oddworld: Abe's Oddysee®,['Adventure']
40,Ankh 2: Heart of Osiris,['Adventure']


In [33]:
# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UserForGenre( genero : str ): 
    """Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista 
    de la acumulación de horas jugadas por año de lanzamiento."""
    
    df_items = pd.read_parquet("M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.parquet", columns=['user_id', 'item_id', 'playtime_forever']) 
    df_steam = pd.read_csv("M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"genres"])
  
    genero = genero.lower()
    genero = genero.title()

    # Filtrar el DataFrame df_steam para obtener los juegos del género especificado
    df_steam_filtrado  = df_steam[df_steam['genres'].apply(lambda x: genero in x)]
    del df_steam
    # Unir df_steam_filtrado con df_items basado en 'item_id' para obtener los usuarios y horas jugadas para los juegos del género 
    df_items.set_index('item_id', inplace=True)
    df_steam_filtrado.set_index('item_id', inplace=True)
    df_combinado = df_items.join(df_steam_filtrado[['year']], on='item_id', how='inner')
    # df_combinado = pd.merge(df_items[['user_id', 'playtime_forever']], df_steam_filtrado[['year']], left_index=True, right_index=True, how='inner')
    # df_combinado = pd.merge(df_steam_filtrado[['item_id', 'year',"genres"]], df_items[['user_id', 'item_id', 'playtime_forever']], on='item_id', how='inner')
    del df_steam_filtrado,  df_items

    # Agrupar por 'user_id' y sumar las horas jugadas para cada usuario
    horas_por_usuario = df_combinado.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Obtener el usuario con más horas jugadas
    usuario_mas_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax(), 'user_id']

    # Agrupar por año y sumar las horas jugadas para cada año
    horas_por_año_usuario = df_combinado[df_combinado['user_id'] == usuario_mas_horas].groupby('year')['playtime_forever'].sum().reset_index()

    # Convertir a la lista deseada de acumulación de horas jugadas por año
    dicc_horas_por_year = [{'Año': row['year'], 'Horas': row['playtime_forever']} for index, row in horas_por_año_usuario.iterrows()]

    respuesta = {
    f"Usuario con más horas jugadas para Género {genero}": usuario_mas_horas,
    "Horas jugadas": dicc_horas_por_year }

    return respuesta



<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Flor Pedraza\AppData\Local\Temp\ipykernel_16520\1661750132.py:7: SyntaxWarning: invalid escape sequence '\D'
  df_items = pd.read_parquet("M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\items_post_etl.parquet", columns=['user_id', 'item_id', 'playtime_forever'])
C:\Users\Flor Pedraza\AppData\Local\Temp\ipykernel_16520\1661750132.py:8: SyntaxWarning: invalid escape sequence '\D'
  df_steam = pd.read_csv("M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'year',"genres"])


In [35]:
UserForGenre("Casual") 

{'Usuario con más horas jugadas para Género Casual': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 0.0, 'Horas': 0.0},
  {'Año': 1999.0, 'Horas': 0.0},
  {'Año': 2002.0, 'Horas': 0.0},
  {'Año': 2004.0, 'Horas': 6985.0},
  {'Año': 2007.0, 'Horas': 0.0},
  {'Año': 2008.0, 'Horas': 1241.0},
  {'Año': 2009.0, 'Horas': 2870.0},
  {'Año': 2010.0, 'Horas': 12066.0},
  {'Año': 2011.0, 'Horas': 81201.0},
  {'Año': 2012.0, 'Horas': 29722.0},
  {'Año': 2013.0, 'Horas': 107965.0},
  {'Año': 2014.0, 'Horas': 122364.0},
  {'Año': 2015.0, 'Horas': 362201.0},
  {'Año': 2016.0, 'Horas': 498260.0},
  {'Año': 2017.0, 'Horas': 58.0}]}